# Cours de Python - Utilisation de l'API IIIF et manipulation de données 2

---

## Table des Matières

1. [Révisions](#révisions)
    - [Structure des manifestes](#structure-des-manifestes)
        - [Les principales composantes](#les-principales-composantes)
        - [Les canvas](#les-canvas)
        - [Trouver l'URL de l'image](#trouver-lurl-de-limage)
    - [Bibliothèque JSON](#bibliothèque-json)
        - [Rappel sur la bibliothèque](#rappel-sur-la-bibliothèque)
        - [Ouvrir un fichier JSON](#ouvrir-un-fichier-json)
        - [Écrire et enregistrer un fichier JSON](#écrire-et-enregistrer-un-fichier-json)
    - [La bibliothèque `pathlib`](#la-bibliothèque-pathlib)
        - [Intérêt de `pathlib`](#intérêt-de-pathlib)
        - [Création de chemins de fichiers avec `pathlib`](#création-de-chemins-de-fichiers-avec-pathlib)
        - [Création de dossier](#création-de-dossier)
    - [Les décorateurs](#Les-décorateurs)
        - [Qu'est-ce qu'un décorateur en Python ?](#Qu'est-ce-qu'un-décorateur-en-Python-?)
        - [Pourquoi utiliser des décorateurs ?](#Pourquoi-utiliser-des-décorateurs-?)
        - [Comment fonctionnent les décorateurs ?](#Comment-fonctionnent-les-décorateurs-?)
2. [Télécharger des données avec la bibliothèque `requests`](#télécharger-des-données-avec-la-bibliothèque-requests)
    - [Intérêt de la bibliothèque](#intérêt-de-la-bibliothèque)
    - [Manipulation des codes et gestion des exceptions](#manipulation-des-codes-et-gestion-des-exceptions)
    - [Méthode `get` et gérer les `status_code`](#méthode-get-et-gérer-les-status_code)
    - [Téléchargement avec `requests`](#téléchargement-avec-requests)
3. [Manipuler des images](#manipuler-des-images)
    - [La bibliothèque `PIL`](#la-bibliothèque-pil)
    - [Manipulation de base d'une image](#manipulation-de-base-dune-image)
    - [Récupérer les données des images](#récupérer-les-données-des-images)

---

## Révisions

### Structure des manifestes

#### Les principales composantes

Un **manifeste** est un document structuré qui décrit une ressource numérique complexe. Il est souvent utilisé pour représenter des collections d'images, de textes ou de médias. Dans le contexte de l'histoire numérique, les manifestes facilitent l'accès et l'interopérabilité des ressources historiques en ligne.

- **Identifiant unique (`@id`)** : Un URL unique qui identifie le manifeste.
- **Type (`@type`)** : Indique le type de ressource (par exemple, "sc:Manifest").
- **Métadonnées (`metadata`)** : Informations sur la ressource, telles que le titre, le créateur, la date, etc.
- **Séquences (`sequences`)** : Liste des séquences de contenu, généralement une seule séquence nommée "normal".
- **Canvases (`canvases`)** : Représente les unités de contenu individuelles, comme les pages d'un livre, les folios d'un manuscrit ou les images d'une collection.

Exemple d'un manifeste simplifié au format JSON :


```json
{
  "@context": "http://iiif.io/api/presentation/2/context.json",
  "@id": "http://example.org/manifest.json",
  "@type": "sc:Manifest",
  "label": "Titre de la ressource",
  "metadata": [
    {
      "label": "Créateur",
      "value": "Nom du créateur"
    }
  ],
  "sequences": [
    {
      "@type": "sc:Sequence",
      "canvases": [
        {
          "@id": "http://example.org/canvas/p1",
          "@type": "sc:Canvas",
          "label": "Page 1",
          "images": [
            {
              "@type": "oa:Annotation",
              "motivation": "sc:painting",
              "resource": {
                "@id": "http://example.org/image/p1/full/full/0/default.jpg",
                "@type": "dctypes:Image",
                "format": "image/jpeg",
                "width": 1000,
                "height": 1500
              },
              "on": "http://example.org/canvas/p1"
            }
          ]
        }
      ]
    }
  ]
}
```

#### Les canvas

Un **canvas** représente une unité de contenu, comme une page ou une image individuelle. Il est essentiel pour structurer et naviguer à travers une collection ou un document complexe.

Propriétés d'un canvas :

- **Identifiant (`@id`)** : URL unique du canvas.
- **Type (`@type`)** : Généralement "sc:Canvas".
- **Label** : Nom ou titre du canvas.
- **Dimensions** : Largeur et hauteur du canvas.
- **Images** : Liste des annotations d'images associées au canvas. L'annotation dans le contexte de IIIF contient des informations sur l'image, telles que :
    - @type : Type de l'annotation, par exemple, "Annotation".
    - motivation : But de l'annotation. Pour une image, c'est souvent "sc:painting", ce qui signifie qu'elle est peinte ou placée sur le canvas.
    - resource : La ressource multimédia, souvent un lien vers l'image elle-même. Il peut contenir des détails sur la taille de l'image et des informations de type MIME (comme image/jpeg).
    - on : Référence au canvas sur lequel l'image doit être affichée. Cela associe l'image à un endroit spécifique sur ce canvas.

```json
{
    "@id": "https://www.e-codices.unifr.ch/metadata/iiif/bbb-0318/canvas/bbb-0318_001r.json",
    "@type": "sc:Canvas",
    "label": "1r",
    "height": 6496,
    "width": 4872,
    "images": [
        {
            "@id": "https://www.e-codices.unifr.ch/metadata/iiif/bbb-0318/annotation/bbb-0318_001r.json",
            "@type": "oa:Annotation",
            "motivation": "sc:painting",
            "on": "https://www.e-codices.unifr.ch/metadata/iiif/bbb-0318/canvas/bbb-0318_001r.json",
            "resource": {
                "@id": "https://www.e-codices.unifr.ch/loris/bbb/bbb-0318/bbb-0318_001r.jp2/full/full/0/default/jpg",
                "@type": "dctypes:Image",
                "format": "image/jpeg",
                "height": 6496,
                "width": 4872,
                "service": {
                    "@context": "http://iiif.io/api/image/2/context.json",
                    "@id": "https://www.e-codices.unifr.ch/loris/bbb/bbb-0318/bbb-0318_001r.jp2",
                    "profile": "http://iiif.io/api/image/2/level2.json"
                }
            }
        }
    ]
}

#### Trouver l'URL de l'image

Pour extraire l'URL d'une image spécifique à partir d'un manifeste, vous devez naviguer à travers la structure imbriquée du JSON.

Exemple en Python :

```python
iiif_manifest = open_json('Berne_318.json')


# Deux approches pour parcourir un manifest: 

# Comme un dictionaire en appelant les clés et les valeurs

"""for key, value in iiif_manifest.items():
    print(key)
"""
# En parcourant simplement les clés
for item in iiif_manifest:
    print(item)

# Avantage de la deuxième pour taper sur les clés qui nous intéresse

id = iiif_manifest['@id']
print(id)

"""for i, canvas in enumerate(iiif_manifest['sequences'][0]['canvases']):
    manifestURL = iiif_manifest['@id']
    canvasId = canvas['@id']
    urlImage = canvas['images'][0]['resource']['@id']
    imageLabel = canvas['label']
    imageWidthAsDeclared = canvas['width']
    imageHeightAsDeclared = canvas['height']
    image_format = urlImage.split('.')[-1]
    image_id = f"{i+1}"
    htmlCode = requests.get(urlImage).status_code
    path_to_store_image = os.path.join(dataset_folder, ms_base_name + '_' + image_id + '.' + image_format)"""

```

In [34]:
import json

def find_url_img(iiif_manifest):
    with open(iiif_manifest, 'r') as file:
        manifest = json.load(file)

    # Accéder au premier canvas
    canvas = manifest['sequences'][0]['canvases'][0]

    # Accéder à l'annotation de l'image
    annotation = canvas['images'][0]

    # URL de l'image
    image_url = annotation['resource']['@id']

    print("URL de l'image :", image_url)

#### Exercice

- Chargez l'un des manifestes suivants :

  - **BVMM** : `https://bvmm.irht.cnrs.fr/iiif/22470/manifest`
  - **e-codices** : `https://www.e-codices.unifr.ch/metadata/iiif/bbb-0318/manifest.json`

- Affichez le titre et les métadonnées du manifeste.

In [ ]:
# Votre code ici

---

### Bibliothèque JSON

#### Rappel sur la bibliothèque

La bibliothèque `json` est intégrée à Python et permet de travailler facilement avec des données au format JSON.

- **JSON** (JavaScript Object Notation) est un format léger d'échange de données, facile à lire et à écrire pour les humains, et facile à analyser et générer pour les machines.

#### Ouvrir un fichier JSON

Pour lire un fichier JSON en Python :

```python
import json

with open('data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

print(data)
```

Gestion des exceptions :

```python
try:
    with open('data.json', 'r', encoding='utf-8') as file:
        data = json.load(file)
except json.JSONDecodeError as e:
    print("Erreur lors du décodage JSON :", e)
except FileNotFoundError:
    print("Fichier non trouvé.")
```

#### Écrire et enregistrer un fichier JSON

Pour écrire des données Python dans un fichier JSON :

```python
import json

data = {
    'name': 'Historien',
    'projets': ['numérisation', 'analyse']
}

with open('output.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, indent=4, ensure_ascii=False)
```

Notes :

- `indent=4` : pour une indentation de 4 espaces, améliore la lisibilité.
- `ensure_ascii=False` : pour conserver les caractères spéciaux (accents, etc.).

---

### La bibliothèque `pathlib`

#### Intérêt de `pathlib`

La bibliothèque `pathlib` permet de **manipuler les fichiers et répertoires** de manière moderne et intuitive :

- Gestion des chemins, fichiers et dossiers via des objets `Path`.
- Navigation, création et suppression d’éléments du système de fichiers.
- Code **plus lisible, plus sûr et compatible** entre Windows, macOS et Linux.

💡 `pathlib` remplace avantageusement les fonctions de `os` et `os.path` pour toutes les opérations sur les chemins et fichiers.

#### Création de chemins de fichiers avec `pathlib`

Construction de chemins de manière **simple et indépendante du système** :

```python
from pathlib import Path

# Chemin relatif
chemin = Path("dossier") / "sous_dossier" / "fichier.txt"
print(chemin)

# Chemin absolu
chemin_absolu = chemin.resolve()
print(chemin_absolu)
```

✅ Plus besoin de `os.path.join` :
l’opérateur `/` gère automatiquement les séparateurs selon le système d’exploitation.

Fonctions utiles avec `pathlib` :

- `chemin.exists()` : Vérifie si le chemin existe.
- `chemin.parent` : Renvoie le répertoire parent.
- `chemin.name` : Renvoie le nom du fichier (avec extension).
- `chemin.stem` : Nom du fichier sans extension.
- `chemin.suffix` : Extension du fichier (ex. .jpg).

💡 Avec `pathlib`, ces propriétés et méthodes sont directement accessibles sur les objets `Path`, ce qui rend le code plus lisible et naturel.

#### Création de dossier

Créer un dossier en toute simplicité :

```python
from pathlib import Path

dossier = Path("nouveau_dossier")

if not dossier.exists():
    dossier.mkdir()
    print(f"Dossier '{dossier}' créé.")
else:
    print(f"Dossier '{dossier}' existe déjà.")
```

✅ Avantages

- Plus lisible et pythonique.
- `Path.mkdir()` gère directement la création du dossier.

Possibilité d’ajouter :

```python
dossier.mkdir(parents=True, exist_ok=True)
```

- crée toute l’arborescence si nécessaire `parents=True`
- n’écrase pas les dossiers déjà existants. `exist_ok=True`

Créer des répertoires imbriqués :

```python
from pathlib import Path

dossiers = Path("dossier1") / "dossier2" / "dossier3"

if not dossiers.exists():
    dossiers.mkdir(parents=True)
    print(f"Dossiers '{dossiers}' créés.")
else:
    print(f"Dossiers '{dossiers}' existent déjà.")
```
✅ Avantages :

- Le `/` gère automatiquement la construction du chemin.

- L’option `parents=True` crée tous les dossiers intermédiaires si nécessaire.

- On peut ajouter `exist_ok=True` pour éviter les erreurs si les dossiers existent déjà.

Gestion des exceptions avec `pathlib` :

```python
from pathlib import Path

dossiers = Path("dossier1") / "dossier2" / "dossier3"

try:
    dossiers.mkdir(parents=True, exist_ok=True)
except Exception as e:
    print(f"Erreur lors de la création des dossiers : {e}")
```

💡 Plus simplement (et recommandé dans la plupart des cas) :

```python
dossiers.mkdir(parents=True, exist_ok=True)
```

✅ `pathlib` gère déjà les cas d’existences de dossiers via `exist_ok=True`, plus besoin de manipuler manuellement les erreurs comme avec `os.errno`.

---

### Les décorateurs

#### Qu'est-ce qu'un décorateur en Python ?

Un **décorateur** est une fonction qui prend en entrée une autre fonction et retourne une nouvelle fonction avec un comportement modifié ou étendu.

- **But** : Modifier le comportement d'une fonction sans changer son code source.

#### Pourquoi utiliser des décorateurs ?

- **Réutilisation du code** : Appliquer le même comportement à plusieurs fonctions.
- **Séparation des préoccupations** : Isoler le code additionnel du code principal.

#### Comment fonctionnent les décorateurs ?

Créons un décorateur qui journalise les appels de fonctions, ce qui peut être utile pour suivre les manipulations sur des documents historiques.

In [ ]:
def journaliser(fonction):
    def nouvelle_fonction(*args, **kwargs):
        print(f"Appel de la fonction '{fonction.__name__}' avec les arguments {args} {kwargs}")
        resultat = fonction(*args, **kwargs)
        print(f"Fin de l'appel de '{fonction.__name__}'")
        return resultat
    return nouvelle_fonction

# Utilisation du décorateur
@journaliser
def analyser_document(titre, date):
    print(f"Analyse du document '{titre}' datant de {date}.")

# Appel de la fonction
analyser_document("The Aberdeen bestiary", "1200")

---

## Télécharger des données avec la bibliothèque `requests`

### Intérêt de la bibliothèque

La bibliothèque `requests` simplifie les requêtes HTTP en Python :

- Plus conviviale que le module intégré `urllib`.
- Gère les sessions, les cookies, l'authentification, etc.
- Bien documentée et largement utilisée.

Installation :

```bash
pip install requests
```

### Manipulation des codes et gestion des exceptions

Comprendre les codes de statut HTTP :

- **200** : OK (succès).
- **404** : Not Found (non trouvé).
- **429** : Too Many Requests
- **500** : Internal Server Error (erreur serveur).

Exemple :

In [35]:
import requests

response = requests.get('https://www.e-codices.unifr.ch/metadata/iiif/bbb-0318/manifest.json')

print("Code de statut :", response.status_code)

Code de statut : 200


### Méthode `get` et gérer les `status_code`

Gestion des erreurs :

```python
if response.status_code == 200:
    data = response.json()
else:
    print(f"Erreur {response.status_code} lors de la requête.")
```

Exceptions de `requests` :

```python
import requests

try:
    response = requests.get('https://api.example.com/data')
    response.raise_for_status()  # Lève une exception pour les codes d'erreur HTTP
    data = response.json()
except requests.exceptions.HTTPError as errh:
    print("Erreur HTTP :", errh)
except requests.exceptions.ConnectionError as errc:
    print("Erreur de connexion :", errc)
except requests.exceptions.Timeout as errt:
    print("Délai d'attente dépassé :", errt)
except requests.exceptions.RequestException as err:
    print("Erreur lors de la requête :", err)
```

### Téléchargement avec `requests'

#### Télécharger un manifeste depuis une URL :

In [36]:
import json
import requests
import os

def download_manifest(manifest_url, output_folder, ms_name):
    # URL du manifeste IIIF
    manifest_url = 'https://www.e-codices.unifr.ch/metadata/iiif/bbb-0318/manifest.json'

    # Télécharger le manifeste
    response = requests.get(manifest_url)

    # Vérifier que la requête s'est bien passée
    if response.status_code == 200:
        manifest = response.json()
        print("Manifeste chargé avec succès.")
        manifest_path = os.path.join(output_folder, ms_name + '_manifest.json')
        
        # Save the manifest.json file in the destination folder
        with open(manifest_path, 'wb') as f:
            f.write(response.content)
        print(f"Manifeste téléchargé dans : {os.path.join(output_folder, ms_name + '_manifest.json')}")
    else:
        print(f"Erreur lors du chargement du manifeste : {response.status_code}")

#### Téléchargement d'image

Télécharger une image depuis une URL :

In [37]:
import requests

url = 'https://www.e-codices.unifr.ch/loris/bbb/bbb-0318/bbb-0318_001r.jp2/full/full/0/default/jpg'

try:
    response = requests.get(url)
    response.raise_for_status()
    
    # Vérifier le type de contenu
    if url.endswith('jpg'):
        with open('image.jpg', 'wb') as file:
            file.write(response.content)
        print("Image téléchargée avec succès.")
    else:
        print("URL ne pointe pas vers une image.")
except requests.exceptions.RequestException as e:
    print(f"Erreur lors du téléchargement : {e}")


Image téléchargée avec succès.


## Manipuler des images

### La bibliothèque PIL

#### Présentation de PIL/Pillow

- **PIL** : Python Imaging Library, bibliothèque historique pour la manipulation d'images.
- **Pillow** : Fork de PIL, activement maintenu et compatible avec les versions récentes de Python.

Installation :

```bash
pip install Pillow
```

Importation :

```python
from PIL import Image
```

### Manipulation de base d'une image

#### Ouvrir et afficher une image

In [ ]:
from PIL import Image

image = Image.open('PATHTODOWLOAD/image.jpg')
image.show()

"from PIL import Image\n\nimage = Image.open('PATHTODOWLOAD/image.jpg')\nimage.show()"

#### Redimensionner une image

In [ ]:
nouvelle_image = image.resize((800, 600))
nouvelle_image.save('image_redimensionnee.jpg')

#### Rotation

In [ ]:
image_rotated = image.rotate(90)
image_rotated.save('image_rotated.jpg')

#### Conversion de format

In [ ]:
image.convert('L').save('image_grayscale.jpg')

#### Recadrage

In [ ]:
box = (200, 900, 2400, 3500)  # (left, upper, right, lower)
image_cropped = image.crop(box)
image_cropped.save('image_cropped.jpg')

### Récupérer les données des images

#### Accéder aux métadonnées

Informations de base :

In [ ]:
print(f"Format : {image.format}")
print(f"Taille : {image.size}")
print(f"Absoluter path : {image.filename}")

Format : JPEG
Taille : (4872, 6496)
Absoluter path : /Users/marioncharpier/Documents/Cours_ENC/Cours_6_Utilisation de l’API IIIF et manipulation de données 2/image.jpg


#### Exercice

- Chargez une image  de votre choix.
- Effectuez les opérations suivantes :
  - Redimensionnez l'image pour qu'elle ait une largeur de 800 pixels tout en conservant le ratio.
  - Appliquez un filtre de netteté pour améliorer la lisibilité (parcourir la doc : https://pillow.readthedocs.io/en/stable/reference/ImageFilter.html).

In [ ]:
# Votre code ici

---

### Conclusion

En maîtrisant ces bibliothèques et techniques, vous serez en mesure de :

- Manipuler des données structurées (JSON).
- Interagir avec le système de fichiers.
- Télécharger des ressources en ligne de manière efficace et sécurisée.
- Traiter et analyser des images pour vos projets en histoire numérique.

Ces compétences sont essentielles pour analyser, conserver et diffuser le patrimoine historique dans le contexte numérique actuel.

---

Vous pouvez utiliser ce notebook comme base pour vos propres projets, en adaptant et en étendant les exemples fournis.

---